Import packages

In [15]:
from utils import *
import numpy as np
import torch
from valda.valuation import DataValuation

Initialize the parameters

In [16]:
train_size = 1000
train_start = 800
train_size_batch = 200
dev_size = 100
test_size = 300
np.random.seed(0)
torch.manual_seed(0)

Load the MNIST dataset and split it into training and test sets.

In [17]:
X_train, y_train_clean, X_dev, y_dev, X_test, y_test = load_data(train_size, dev_size, test_size)
y_train, flip_indices = flip_labels(y_train_clean, flip_fraction=0.3)

Divide the train set into batches and run the valuation.

In [18]:
X_train_batch = X_train[train_start:train_start+train_size_batch]
y_train_batch = y_train[train_start:train_start+train_size_batch]

In [19]:
# Define a DataValuation instance
dv = DataValuation(X_train_batch, y_train_batch, X_dev, y_dev)

# Run with the default setting:
# 1. Logistic regression as the classifier
# 2. Prediction accuracy as the value measurement
vals_tmc = dv.estimate(method="tmc-shapley")

100%|██████████| 500/500 [18:39<00:00,  2.24s/it]


Get the indices of the data points with values < 0

In [20]:
min_indices_batch = [key for key, value in vals_tmc.items() if value <= 0]
min_indices_batch = np.array(min_indices_batch) + train_start

Append the indices to a file

In [21]:
if not os.path.exists('./data'):
    os.makedirs('./data')

file_path = f'./data/min_indices_{str(train_start).zfill(3)}_{str(train_start+train_size_batch-1).zfill(3)}.npy'
np.save(file_path, min_indices_batch)